In [ ]:
#This code works and was originally used to construct the connectivity matrices on Farnam (7/12/2022). However, the most up to date 
#dataset for MDD TRT was processed by Dylan at the NIH
#We also used this code to generate the Schaefer connectomes for the revision

In [1]:
from pathlib import Path
from nilearn import datasets, image, plotting, masking
from bids import BIDSLayout
from ast import literal_eval
from io import StringIO
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast


%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

/tmp/ipykernel_23124/2479951448.py:19: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [41]:
project_root = Path('/home/ccc98/Documents/CATD-ReliabilityAnalysis')
ts_dir = project_root / 'data' / 'CATD' / 'derivatives' / 'ts_extract_allmodels'
corr_dir = project_root / 'data' / 'processed' / 'connectomes'
database_path = project_root / 'references' / 'rest_paths.csv'
rest_paths = pd.read_csv(database_path)
rest_paths = rest_paths[[col for col in rest_paths.columns if "path" not in col]]
rest_paths['entities'] = rest_paths['entities'].apply(lambda x: literal_eval(x))

In [42]:
#Make sure to check for run = 1.0 vs. run = 1

def make_path(ents, updates, pattern, mp_dir, check_exist=False):
    mp_ents = ents.copy()
    mp_ents.update(updates)
    mp_file = mp_dir / ('sub-'+str(mp_ents['subject'])) / ('ses-'+mp_ents['session']) / ('sub-'+str(mp_ents['subject'])+'_ses-'+mp_ents['session']+'_task-'+mp_ents['task']+'_run-'+str(mp_ents['run'])+'_space-'+mp_ents['space']+'_desc-'+mp_ents['desc']+'_'+mp_ents['suffix']+'.'+mp_ents['extension'])
    mp_file.parent.mkdir(exist_ok=True, parents=True)
    if check_exist and not mp_file.exists():
        print(f"FileNotFound: {mp_file.as_posix()}")
#         raise FileNotFoundError(mp_file.as_posix())
    return mp_file

In [43]:
#Make sure to check for run = 1.0 vs. run = 1

# ts_pattern = 'sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_run-{run}.0_space-{space}_desc-{desc}_{suffix}.{extension}'
# notd_basc_nodlpfc_updates = {'task':'rest','desc':'notdBasc122_nodlpfc', 'suffix':'timeseries', 'extension':'csv'}
# rest_paths['notd_basc_nodlpfc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
#                                                             notd_basc_nodlpfc_updates,
#                                                             ts_pattern,
#                                                             ts_dir,
#                                                             check_exist=True))

notd_basc_updates = {'task':'rest','desc':'notdBasc122', 'suffix':'timeseries', 'extension':'csv'}
rest_paths['notd_basc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            notd_basc_updates,
                                                            ts_pattern,
                                                            ts_dir,
                                                            check_exist=True))

notdBasc122_wSchaeferdlpfc_updates = {'task':'rest','desc':'notdBasc122wSchaeferdlpfc', 'suffix':'timeseries', 'extension':'csv'}
rest_paths['notd_basc_wSchaeferdlpfc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            notd_basc122_wSchaeferdlpfc_updates,
                                                            ts_pattern,
                                                            ts_dir,
                                                            check_exist=True))

notdSchaefer_updates = {'task':'rest','desc':'notdSchaefer400', 'suffix':'timeseries', 'extension':'csv'}
rest_paths['notdSchaefer_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            notdSchaefer_updates,
                                                            ts_pattern,
                                                            ts_dir,
                                                            check_exist=True))

notdSchaefersc_updates = {'task':'rest','desc':'notdSchaefer400sc', 'suffix':'timeseries', 'extension':'csv'}
rest_paths['notdSchaefersc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            notdSchaefersc_updates,
                                                            ts_pattern,
                                                            ts_dir,
                                                            check_exist=True))

notdSchaeferscwdlpfc_updates = {'task':'rest','desc':'notdSchaefer400scwdlpfc', 'suffix':'timeseries', 'extension':'csv'}
rest_paths['notdSchaeferscwdlpfc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            notdSchaeferscwdlpfc_updates,
                                                            ts_pattern,
                                                            ts_dir,
                                                            check_exist=True))

notdSchaeferwdlpfc_updates = {'task':'rest','desc':'notdSchaefer400wdlpfc', 'suffix':'timeseries', 'extension':'csv'}
rest_paths['notdSchaeferwdlpfc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            notdSchaeferwdlpfc_updates,
                                                            ts_pattern,
                                                            ts_dir,
                                                            check_exist=True))

rest_corr_notdSchaefer_updates = {'task':'rest','desc':'notdSchaefer', 'suffix':'correlation', 'extension':'csv'}
rest_paths['rest_corr_notdSchaefer_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            rest_corr_notdSchaefer_updates,
                                                            ts_pattern,
                                                            corr_dir,
                                                            check_exist=False))

rest_corr_notd_basc_updates = {'task':'rest','desc':'notdBasc122', 'suffix':'correlation', 'extension':'csv'}
rest_paths['rest_corr_notd_basc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            rest_corr_notd_basc_updates,
                                                            ts_pattern,
                                                            corr_dir,
                                                            check_exist=False))

rest_corr_notdSchaefersc_updates = {'task':'rest','desc':'notdSchaefersc', 'suffix':'correlation', 'extension':'csv'}
rest_paths['rest_corr_notdSchaefersc_path'] = rest_paths.entities.apply(lambda x: make_path(x,
                                                            rest_corr_notdSchaefersc_updates,
                                                            ts_pattern,
                                                            corr_dir,
                                                            check_exist=False))




In [18]:
#Running on Farnam
atlas_path = project_root / 'data/processed/atlases/basc_122_nearest_atlas.nii.gz'
regions_extracted_img = image.load_img(str(atlas_path))

In [44]:
rest_paths_back = rest_paths.copy()

In [ ]:
rest_paths.loc[rest_paths.exclude == True,:]

In [67]:
#run correlations, drop subjects with too few TRs
#first checks if correlation matrices already exist. To overwrite existing matrices, comment out the assertion
model = "notdSchaefersc"
excluded_scans = pd.DataFrame()
excluded_indices = []
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
for i,row in rest_paths.iterrows():
    tr_path = row[model+"_path"]
    cm_path = row["rest_corr_"+model+"_path"]
    try:
        assert(~cm_path.exists())
        correlation_matrix = correlation_measure.fit_transform([pd.read_csv(tr_path,header=None).values])[0]
        pd.DataFrame(correlation_matrix).to_csv(cm_path, header=None, index=False)
    except:
        excluded_scans = pd.concat([excluded_scans, row])
        excluded_indices.append(i)
rest_paths = rest_paths.drop(excluded_indices)
print(len(rest_paths_back), len(rest_paths), len(excluded_indices))
excluded_scans = rest_paths_back.iloc[excluded_indices]
excluded_scans

550 550 0


,subject,run,suffix,session,task,note_bbr,usable_bbr,dn_action_bbr,note_sdc,usable_sdc,dn_action_sdc,usable,space,datatype,extension,motion_per_tr,motion_per_tr_pc,max_fd,max_fd_pc,pct_censored,nv,ms,has_phys,exclude,session_id,sex,group,age,scanner,inpatient,c_ksadsdx_offset,c_ksadsdx_dx_detailed,s_mfq_offset,s_mfq_tot,s_mfq_complete,s_shaps_offset,s_shaps_tot,s_shaps_binary_tot,s_shaps_complete,s_ari1w_offset,s_ari1w_tot,s_ari1w_complete,s_scared_offset,s_scared_tot,s_scared_complete,antidepressants,antipsychotics,anticonvulsants,ADHD_medication,antianxiety,other,no_meds,med_source,dropout,above_mfq_thresh,sub_ses,acq_dat,entities,file,subsample,n_sessions,echo1_et,echo2_et,echo3_et,echo4_et,echo1sample_exists,boldmask_exists,tedana_failed,sample_weight,notd_basc_path,notd_basc_wSchaeferdlpfc_path,notdSchaefer_path,notdSchaefersc_path,notdSchaeferscwdlpfc_path,notdSchaeferwdlpfc_path,rest_corr_notdSchaefer_path,rest_corr_notdBasc122_path,rest_corr_notdSchaefersc_path


In [56]:
rest_paths.to_csv(project_root / 'references' / 'rest_df.csv',index=False)

In [22]:
rest_paths_light = rest_paths[['subject','session','group']]

In [23]:
rest_paths_light.to_csv(project_root / 'references' / 'rest_df_v21_1_light.csv',index=False)